In [ ]:
from getdatatestbed import getDataFRF
import datetime as DT 
import numpy as np
from matplotlib import pyplot as plt 
%matplotlib inline
import netCDF4 as nc 

This script is an example for mike forte to evaluate and look at the regression through time of the entire FRF property.  This as it stands right now is only on the North side, it does a gridded evolution through time of the survey data.  Things to notice are the surveys changed around 1995 (dunes stopped being captured, and profiles lengthened).  the cross-shore migration of sandbars from the shoreline to about 400m before they dissapear

# TODO
- do the same thing on the south side
- do a subtraction of the North and south profiles to highlight the difference (if any) between most offshore profiles. tip: use midpoint normalize to center the RdBu colormap on zero when doing the diffence plot 

# possible problems 
- time as its rounded right now seems to round part of the surveys in to one day and part into another day. this causes some of the data to be excluded om the analysis, it should be more throughly added 

In [ ]:
def roundTime(dt=None, roundTo=60):
   """Round a datetime object to any time lapse in seconds
   dt : datetime.datetime object, default now.
   roundTo : Closest number of seconds to round to, default 1 minute.
   Author: Thierry Husson 2012 - Use it as you want but don't blame me.
   """
   if dt == None : dt = DT.datetime.now()
   seconds = (dt.replace(tzinfo=None) - dt.min).seconds
   rounding = (seconds+roundTo/2) // roundTo * roundTo
   return dt + DT.timedelta(0,rounding-seconds,-dt.microsecond)

import matplotlib.colors as colors

class MidpointNormalize(colors.Normalize):
    """	Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)

    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))

    Examples:
        elev_min=-1000
        elev_max=3000
        mid_val=0

        plt.imshow(ras, cmap=cmap, clim=(elev_min, elev_max), norm=MidpointNormalize(midpoint=mid_val,vmin=elev_min, vmax=elev_max))
        plt.colorbar()
        plt.show()
    """

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

In [ ]:
go  = getDataFRF.getObs(DT.datetime(1900,1,1), DT.datetime(2019,12,1))
survey = go.getBathyTransectFromNC(forceReturnAll=True)
print(survey.keys())
# what are my north and south lines of interest
NorthIdx = survey['profileNumber'] == 1097
southIdx = survey['profileNumber'] == 1
crossShoreMax = 1500   # how far do we want to look in cross-shore 

In [ ]:
# isolate data for Northside line only 
timesNorth = survey['time'][NorthIdx]
xFRFNorth = survey['xFRF'][NorthIdx]
elevationNorth = survey['elevation'][NorthIdx]
surveyNumberNorth = survey['surveyNumber'][NorthIdx]
epochTimesNorth = survey['epochtime'][NorthIdx]
# UniqueSurveyNumbersNorth = np.unique(surveyNumberNorth)
timesNorth = [roundTime(t, roundTo=60*60*24) for t in timesNorth]  #round to nearest Day
epochTimesNorth = nc.date2num(timesNorth, 'seconds since 1970-01-01')
# quick plot to see what we're looking at 
plt.plot(xFRFNorth, elevationNorth,'.')
plt.xlim([0, crossShoreMax])
plt.ylim([-16, 8])


In [ ]:
# now grid data in cross shore
from scipy import interpolate
minPoints4Survey = 10
xOut = np.arange(0,1800)
tOut = np.unique(epochTimesNorth)
#how many unique surveys do i have??
zOutNorth = np.ma.masked_all((tOut.shape[0], xOut.shape[0])) # initalize masked array all values True
for ss, surveyTime in enumerate(tOut):  # ss is index of value surveyNum
    idxSurveyPoints = epochTimesNorth == surveyTime
    print('{} survey points in survey on {}'.format(np.sum(idxSurveyPoints), nc.num2date(surveyTime, 'seconds since 1970-01-01')))
    print('Mean elevations {:.2f}'.format(elevationNorth[idxSurveyPoints].mean()))
    if idxSurveyPoints.sum() > minPoints4Survey:  # sometimes there's not enough points on a day (i don't know why)
        tempf = interpolate.interp1d(xFRFNorth[idxSurveyPoints], elevationNorth[idxSurveyPoints],
                                     bounds_error=False, kind='linear', fill_value='nan')
        zOutNorth[ss] = tempf(xOut)
#     else:
#         print('  deleted ss{}'.format(ss))
#         tOut=np.delete(tOut,ss)

In [ ]:
plt.figure(figsize=(18,6))
plt.pcolormesh(xOut, nc.num2date(tOut, 'seconds since 1970-01-01'), zOutNorth)# , cmap='RdBu', norm=(MidpointNormalize(midpoint=0)))
plt.colorbar()
# notice bar migration patterns 